In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Sample corpus and parameters
corpus = ["I like machine learning", "I enjoy deep", "Coding is fascinating"]
window_size, embedding_dim, lr, epochs = 2, 5, 0.01, 100

# Preprocessing: Tokenize and create vocabulary
words = [word for sentence in corpus for word in sentence.lower().split()]
vocab = sorted(set(words))
word_to_index = {word: idx for idx, word in enumerate(vocab)}
index_to_word = {idx: word for word, idx in word_to_index.items()}

# One-hot encoding
encoder = OneHotEncoder(sparse_output=False)
one_hot_matrix = encoder.fit_transform(np.array(vocab).reshape(-1, 1))

# Generate training data
def generate_training_data(words, window_size):
    X, y = [], []
    for i in range(window_size, len(words) - window_size):
        context = words[i - window_size:i] + words[i + 1:i + window_size + 1]
        target = words[i]
        X.append(np.sum([one_hot_matrix[word_to_index[w]] for w in context], axis=0))
        y.append(one_hot_matrix[word_to_index[target]])
    return np.array(X), np.array(y)

X_train, y_train = generate_training_data(words, window_size)

# Model parameters and layers
W1 = np.random.rand(len(vocab), embedding_dim)
W2 = np.random.rand(embedding_dim, len(vocab))

# Training the CBOW model
for _ in range(epochs):
    for x, target in zip(X_train, y_train):
        # Forward pass
        h = np.dot(x, W1)  # Hidden layer
        u = np.dot(h, W2)  # Output layer
        y_pred = np.exp(u) / np.sum(np.exp(u))  # Softmax activation

        # Calculate error
        e = y_pred - target  # Difference between predicted and actual output

        # Backpropagation and weight updates
        W2 -= lr * np.outer(h, e)  # Update W2
        # Update W1 rows corresponding to context words
        for word_idx in np.where(x != 0)[0]:
            W1[word_idx, :] -= lr * np.dot(e, W2.T)

# Word prediction function
def predict(context_words):
    context_vec = np.sum([one_hot_matrix[word_to_index[word]] for word in context_words], axis=0)
    h = np.dot(context_vec, W1)  # Hidden layer
    u = np.dot(h, W2)  # Output layer
    y_pred = np.exp(u) / np.sum(np.exp(u))  # Softmax activation
    return index_to_word[np.argmax(y_pred)]

# Test prediction
print("Predicted word:", predict(["i", "like", "learning", "i"]))

Predicted word: machine
